# Machine Learning in Python - CIFAR-10 SOLUTION

Based on a CNN CIFAR-10 solution from https://raw.githubusercontent.com/keras-team/keras/master/examples/cifar10_cnn.py

In [ ]:
#import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils.np_utils import to_categorical
from keras.utils.vis_utils import model_to_dot
from keras.optimizers import rmsprop

import matplotlib.pyplot as plt
import random

from sklearn import metrics

import pandas as pd
import numpy as np

from IPython.display import display, HTML, Image
from IPython.display import SVG

from PIL import Image

import os
from random import randint


Prepare the data

In [ ]:
num_classes = 10
classes = {0:"airplane", 1:"automobile", 2:"bird",3:"cat", 4:"deer", 5:"dog", 6:"frog", 7:"horse", 8:"ship", 9:"truck"}

# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
pltsize=2
row_images = 5
col_images = 5
plt.figure(figsize=(col_images*pltsize, row_images*pltsize))

for i in range(row_images * col_images):
    i_rand = random.randint(0, x_train.shape[0])
    plt.subplot(row_images,col_images,i+1)
    plt.axis('off')
    plt.imshow(Image.fromarray(x_train[i_rand]))
    plt.title(str(i_rand) + " " + str(y_train[i_rand][0]) + " (" + classes[y_train[i_rand][0]] + ")")

### Simple Feed-forward model

Reshape the data

In [ ]:
x_train_flat = x_train.reshape(x_train.shape[0], 3072)
x_test_flat = x_test.reshape(x_test.shape[0], 3072)

x_train_flat = x_train_flat.astype('float32')
x_test_flat = x_test_flat.astype('float32')
x_train_flat /= 255
x_test_flat /= 255

y_train = y_train.reshape(y_train.shape[0])
y_test = y_test.reshape(y_test.shape[0])

y_train_wide = to_categorical(y_train, num_classes)
y_test_wide = to_categorical(y_test, num_classes)

In [ ]:
dnn_model = Sequential()
dnn_model.add(Dense(input_dim=3072, units=1000))
dnn_model.add(Activation("relu"))
dnn_model.add(Dense(units=512))
dnn_model.add(Activation("relu"))
dnn_model.add(Dense(units=256))
dnn_model.add(Activation("relu"))
dnn_model.add(Dense(units=10))
dnn_model.add(Activation("softmax"))

dnn_model.summary()

In [ ]:
SVG(model_to_dot(dnn_model).create(prog='dot', format='svg'))

In [ ]:
dnn_model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history_dnn = dnn_model.fit(x_train_flat, y_train_wide, \
          epochs=50, batch_size=128, verbose=1, validation_split=0.2)

In [ ]:
# summarize history for loss
plt.plot(history_dnn.history['loss'])
plt.plot(history_dnn.history['val_loss'])
plt.title('Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

Evaluate the model

In [ ]:
print("******** Training Data ********")
# Make a set of predictions for the training data
y_pred = dnn_model.predict_classes(x_train_flat, batch_size=32)

# Print performance details
print(metrics.classification_report(y_train, y_pred))

# Print confusion matrix
print("Confusion Matrix")
display(pd.crosstab(y_train, y_pred, rownames=['True'], colnames=['Predicted'], margins=True))



print("****** Test Data ********")

# Make a set of predictions for the validation data
y_pred = dnn_model.predict_classes(x_test_flat)

# Print performance details
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix")
display(pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True))

### Convolutional model

Load in the data again as we need to keep it in image shape

In [ ]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

y_train = y_train.reshape(y_train.shape[0])
y_test = y_test.reshape(y_test.shape[0])

y_train_wide = to_categorical(y_train, num_classes)
y_test_wide = to_categorical(y_test, num_classes)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

Define model structure

In [ ]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
cnn_model.add(Activation('relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Conv2D(64, (3, 3)))
cnn_model.add(Activation('relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.25))
cnn_model.add(Conv2D(128, (3, 3), padding='same'))
cnn_model.add(Activation('relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.25))
cnn_model.add(Flatten())
cnn_model.add(Dense(512))
cnn_model.add(Activation('relu'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(num_classes))
cnn_model.add(Activation('softmax'))
cnn_model.summary()

In [ ]:
SVG(model_to_dot(cnn_model).create(prog='dot', format='svg'))

In [ ]:
# initiate RMSprop optimizer
opt = rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
cnn_model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

Train the model

In [ ]:
history_cnn = cnn_model.fit(x_train, y_train_wide,
          batch_size=128,
          epochs=50,
          shuffle=True, validation_split=0.2)

In [ ]:
# summarize history for loss
plt.plot(history_cnn.history['loss'])
plt.plot(history_cnn.history['val_loss'])
plt.title('Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.ylim((-0.1, 1.1))
plt.show()

In [ ]:
print("******** Training Data ********")
# Make a set of predictions for the training data
y_pred = cnn_model.predict_classes(x_train, batch_size=32)

# Print performance details
print(metrics.classification_report(y_train, y_pred))

# Print confusion matrix
print("Confusion Matrix")
display(pd.crosstab(y_train, y_pred, rownames=['True'], colnames=['Predicted'], margins=True))



print("****** Test Data ********")

# Make a set of predictions for the validation data
y_pred = cnn_model.predict_classes(x_test)

# Print performance details
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix")
display(pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True))

### Test Both Models on Unseen Test Data

Score some test data with DNN model

In [ ]:
# Score trained model.
scores = dnn_model.evaluate(x_test_flat, y_test_wide, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
scores

Score some test data with CNN model

In [ ]:
# Score trained model.
scores = cnn_model.evaluate(x_test, y_test_wide, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
scores